In [1]:
import numpy as np
from observables import observables
from k_space import k_space


# Minimal p-Hamiltonian
$p_z,p_x,p_y$-Hamiltonian, with Dirac point in the $p_x,p_y$ bands, gapped by $L_z$
\begin{aligned}
    H = E_{p_z} + k_x \cdot s_x + k_y \cdot s_z + \lambda \cdot s_y
\end{aligned}

In [5]:
s = np.zeros((4,2,2),dtype=complex)
s[0] = np.eye(2)
s[1] = np.array([[0,1],[1,0]])
s[2] = 1j*np.array([[0,-1],[1,0]])
s[3] = np.array([[1,0],[0,-1]])

class dirac_ham:
    def __init__(self,L):
                
        self.l      = L
        self.E_pz   = -10
        self.bra_vec= np.array([[1,0,0],[0,1,0],[0,0,1]])
        self.basis  = np.array([1])
        self.ef     = 0
        self.n_elec = 2
        self.spin   = False
        self.n_orb  = 3
        self.n_bands= 3
        self.ctype  = np.csingle

    def hk(self,k_red):
        d = np.zeros((k_red.shape[0],4))
        d[:,1] = k_red[:,0]
        d[:,3] = k_red[:,1]
        d[:,2] = self.l
        hk_out = np.zeros((k_red.shape[0],3,3),dtype=complex)
        hk_out[:,0,0] = self.E_pz
        hk_out[:,1:,1:] = np.einsum('ij,jkl->ikl',d,s)
        return hk_out
        
    def del_hk(self,k_red):
        d = np.zeros((k_red.shape[0],3,4))
        d[:,0,1] = 1
        d[:,1,3] = 1
        del_hk_out = np.zeros((k_red.shape[0],3,3,3),dtype=complex)
        del_hk_out[:,:,1:,1:] = np.einsum('idj,jkl->idkl',d,s)
        return del_hk_out





In [6]:
ktype  = "path"
kbasis = "red"
vecs   = np.array([[-1,0,0],[1,0,0]])
npoints = 101
bra_vec = np.array([[1,0,0],[0,1,0],[0,0,1]])

K_space = k_space(ktype,kbasis,vecs,bra_vec,npoints)

In [7]:
l=0.2
ham = dirac_ham(l)
op_types = ["L"]
op_types_k = ["BC","BC_L"]
Observables = observables(ham,K_space,op_types,op_types_k)
Observables.calculate_ops()

Initializing k-independent operator L.
Inititalizing k-dependent operator BC.
Inititalizing k-dependent operator BC_L.
Calculating operators on the given k-space...
Diagonalizing all k-points in parallel.
Time for running H(k) FT: 9.083747863769531e-05
Time for diagonalizing H(k): 0.0003409385681152344
Time for calculating expectation value of operator L: 0.0029840469360351562
Time for calculating expectation value of operator BC: 0.0014119148254394531
Time for calculating expectation value of operator BC_L: 0.01288914680480957
Shifting eigenvalues w.r.t. Fermi level...
Running post-processing for operator L.
Running post-processing for operator BC.
Running post-processing for operator BC_L.
Writing eigenvalues output.
Writing output for operator L.
Writing output for operator BC.
Writing output for operator BC_L.
Writing band-integrated output for operator BC.
Writing band-integrated output for operator BC_L.


# Calculate the coupling strength to optical fields
Here we follow the approach given in https://journals.aps.org/prl/pdf/10.1103/PhysRevLett.108.196802 given by
\begin{aligned}
P_\alpha(k)=\langle u_c(k) |\frac{dH}{dk_\alpha}| u_v(k)\rangle
\end{aligned}

In [8]:
ham = dirac_ham(l)
k = np.zeros((1,3))
evals,evecs = np.linalg.eigh(ham.hk(k)[0])
print(evals)
h_del_k = ham.del_hk(k)[0]
print(evecs.shape)
p = np.einsum("j,xjk,k->x",evecs[:,2].conjugate(),h_del_k,evecs[:,1])
print(p)

[-10.   -0.2   0.2]
(3, 3)
[0.-1.j 1.+0.j 0.+0.j]


# Calculation of the $z$-component of the valence BC by hand at k=0

In [10]:
k = np.array([[0,0,0]])
hk = ham.hk(k)
evals,evecs = np.linalg.eigh(hk)

#valene and conduction eigenvectors
v = evecs[:,1:,1]
c = evecs[:,1:,2]
# z-componend of the valence state
vsxc = np.einsum("i,i",v.conjugate()[0],np.einsum("ij,j",s[1],c[0]))
cszv = np.einsum("i,i",c.conjugate()[0],np.einsum("ij,j",s[3],v[0]))

omega_z = 2*1j *vsxc*cszv/(evals[0,1]-evals[0,2])**2
print("Valence band Berry curvature:",omega_z)

Valence band Berry curvature: (-12.499999999999993+0j)


# Minimal p-Hamiltonian
two band Hamiltonian, 3D
\begin{aligned}
    H = \vec{k}\cdot\vec{S}
\end{aligned}

In [11]:
class dirac_ham_3d:
    def __init__(self,SPIN):
                
        self.bra_vec= np.array([[1,0,0],[0,1,0],[0,0,1]])
        self.basis  = np.array([0,0])
        self.ef     = 0
        self.n_elec = 2
        self.spin   = SPIN
        self.n_orb  = 2
        self.n_bands= 2

    def hk(self,k_red):
        d = np.zeros((k_red.shape[0],4))
        d[:,1] = k_red[:,0]
        d[:,2] = k_red[:,1]
        d[:,3] = k_red[:,2]
        hk_out = np.zeros((k_red.shape[0],self.n_orb,self.n_orb),dtype=complex)
        hk_out = np.einsum('ks,sij->kij',d,s)
        return hk_out
        
    def del_hk(self,k_red):
        d = np.zeros((k_red.shape[0],3,4))
        d[:,0,1] = 1
        d[:,1,2] = 1
        d[:,2,3] = 1
        del_hk_out = np.zeros((k_red.shape[0],3,self.n_orb,self.n_orb),dtype=complex)
        del_hk_out = np.einsum('kds,sij->kdij',d,s)
        return del_hk_out

In [12]:
ktype  = "plane"
kbasis = "red"
vecs   = np.array([[0,0,0],[1,0,0],[0,1,0]])
npoints = 101
bra_vec = np.array([[1,0,0],[0,1,0],[0,0,1]])

K_space_map = k_space(ktype,kbasis,vecs,bra_vec,npoints)

ktype  = "sphere"
kbasis = "car"
vecs   = np.array([[0,0,0]])
npoints = 10
r       = 0.1
bra_vec = np.array([[1,0,0],[0,1,0],[0,0,1]])

K_space_sphere = k_space(ktype,kbasis,vecs,bra_vec,npoints,r)


In [13]:
spin=False
ham = dirac_ham_3d(spin)
op_types = []
op_types_k = ["BC"]
Observables = observables(ham,K_space_sphere,op_types,op_types_k,PREFIX="2band_")
Observables.calculate_ops(write=False)
### Integrate the BC
BC = Observables.ops["BC"].val
BC_norm = np.linalg.norm(BC,axis=1)
BC_int = np.sum(BC_norm,axis=0)*r**2/K_space_sphere.k_space_red.shape[0]/np.pi/(2*np.pi)
print(BC_int)


Inititalizing k-dependent operator BC.
Calculating operators on the given k-space...


AttributeError: 'dirac_ham_3d' object has no attribute 'ctype'

# Testing the E_triang operator
At the valley momenta, the projection on the hexagonal sites A/B is given by:
\begin{aligned}
    \langle m| P_{A/B}|m\rangle &= \frac{1}{9}|\sum_{i\in \{A,B\}}\sum_n^3 e^{ikR_n+n\frac{2\pi}{3}m}|^2 \\
    &=\frac{1}{9}|3+2e^{i\phi}+e^{i2\phi}+2e^{-i\phi}+e^{-i2\phi}| \\
    &=\frac{1}{9}(3+2\cos{2\phi}+4\cos{\phi})
\end{aligned}
with $\phi=k-K$ being the momentum deviation from the valley momenta

In [16]:
class hexagonal_p_ham:
    '''
    This Hamiltonian has only a local L_z term for testing the operator "E_triang".
    '''
    def __init__(self,L):
                
        self.l      = L
        self.E_pz   = -10
        self.bra_vec= np.array([[1,0,0],[-1/2,np.sqrt(3)/2,0],[0,0,1]])
        self.basis  = np.array([1])
        self.ef     = 0
        self.n_elec = 2
        self.spin   = False
        self.n_orb  = 3
        self.n_bands= 3
        self.ctype  = np.csingle

    def hk(self,k_red):
        d = np.zeros((k_red.shape[0],4))
        d[:,2] = self.l
        hk_out = np.zeros((k_red.shape[0],3,3),dtype=complex)
        hk_out[:,0,0] = self.E_pz
        hk_out[:,1:,1:] = np.einsum('ij,jkl->ikl',d,s)
        return hk_out

In [17]:
hex_ham = hexagonal_p_ham(0.5)
ktype  = "path"
kbasis = "red"
vecs   = np.array([[0,0,0],[1,1,0]])
npoints = 10
k_path_hex = k_space(ktype,kbasis,vecs,hex_ham.bra_vec,npoints)
observables_hex = observables(hex_ham,k_path_hex,[],["E_triang"],"hex_p_")
observables_hex.calculate_ops()

Inititalizing k-dependent operator E_triang.
Calculating operators on the given k-space...
Diagonalizing all k-points in parallel.
Time for running H(k) FT: 8.106231689453125e-05
Time for diagonalizing H(k): 0.00013494491577148438
Time for calculating expectation value of operator E_triang: 0.0006411075592041016
Shifting eigenvalues w.r.t. Fermi level...
Running post-processing for operator E_triang.
No post-processing.
Writing eigenvalues output.
Writing output for operator E_triang.


In [18]:
### Calculating the expectation value by hand
def sub_latt_loc(k,K):
    phi = 2*np.pi /2* np.sum(k-K[None],axis=1)
    sub_latt = (1/9*(3+2*np.cos(2*phi)+4*np.cos(phi)))
    return sub_latt
#constructive interference at K
K = np.array([1/3,1/3,0])*2
sl_char = sub_latt_loc(k_path_hex.k_space_red,K)
print(sl_char)

[0.00000000e+00 8.59242670e-02 8.59242670e-02 0.00000000e+00
 2.01689719e-01 7.12386014e-01 1.00000000e+00 7.12386014e-01
 2.01689719e-01 4.93432455e-17]


In [19]:
print(observables_hex.ops["E_triang"].val[:,0,2])
np.allclose(sl_char,observables_hex.ops["E_triang"].val[:,0,2])

[2.73910039e-32 8.59242647e-02 8.59242647e-02 6.16297561e-32
 2.01689716e-01 7.12386014e-01 9.99999966e-01 7.12386014e-01
 2.01689716e-01 8.90207614e-32]


True